In [1]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 915.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 36.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [2]:
import pandas as pd

In [4]:
data = pd.read_csv("/content/train.csv",engine="python")
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
data["toxic"].value_counts()

,count
toxic,
0,144277
1,15294


In [6]:
data = data[["comment_text","toxic"]]
data = data[:1000]
data.shape

(1000, 2)

In [7]:
!pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import torch
from transformers import TrainingArguments,Trainer
from transformers import BertTokenizer,BertForSequenceClassification

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model = model.to("cuda")

In [11]:
X = list(data["comment_text"]) # tokenizer expects data in the form of list
y=list(data["toxic"])
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train,padding = True,truncation=True,max_length=512)
X_val_tokenized = tokenizer(X_val,padding = True,truncation = True,max_length=512)

In [12]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [13]:
print(X_train_tokenized["input_ids"][0])

[101, 1000, 28697, 9336, 2586, 2609, 3158, 9305, 2964, 6203, 21605, 18384, 1010, 1045, 2626, 2023, 3602, 2000, 12367, 2017, 2008, 2019, 3559, 2003, 4488, 4490, 1997, 3158, 9305, 2964, 1999, 1996, 4037, 1997, 1996, 1000, 1000, 3060, 2586, 1000, 1000, 16770, 1024, 1013, 1013, 4372, 1012, 16948, 1012, 8917, 1013, 15536, 3211, 1013, 3060, 1035, 2586, 1012, 3531, 1045, 2052, 9120, 2065, 2017, 2071, 2425, 2033, 2040, 1045, 2323, 3189, 2023, 2895, 1029, 1010, 1996, 2604, 5950, 2171, 2003, 16770, 1024, 1013, 1013, 4372, 1012, 16948, 1012, 8917, 1013, 15536, 3211, 1013, 5310, 1024, 14161, 2483, 14876, 1000, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels = None):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    if self.labels:
      item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.encodings['input_ids'])

In [15]:
train_dataset = Dataset(X_train_tokenized,y_train)
val_dataset = Dataset(X_val_tokenized,y_val)

In [16]:
train_dataset[5]

{'input_ids': tensor([  101,  1000,  1996,  2048,  2502,  2477,  2052,  2022, 11091,  2000,
         20410,  2008,  2009,  2001, 27289,  1999,  1996,  4882,  2028,  1006,
         10539,  4573,  1999,  2394,  2074,  2360, 14707,  2078,  5376,  1007,
          1010,  2021,  2144,  2017,  2031,  1996,  2028,  2008,  4627,  7642,
          3989,  1010,  2008,  2064,  2022,  2109,  2000,  3602,  2043,  2009,
          2318,  7642,  3989,  1012,  2065,  2027,  5254,  2045,  2008,  2049,
          1037, 13633,  2013,  1996,  2028,  1011,  2915,  3441,  2138,  1045,
          2064,  1005,  1056,  2424,  2151, 12667,  1010,  2887,  2030,  2394,
          1010,  3784,  2008, 23283,  2008,  1012,  1996,  2060,  2003,  7570,
          3089,  2072,  1005,  1055,  3114,  2005,  5815,  7163,  1011,  6665,
          1012, 18558,  2006,  1996,  8750,  2055,  2054,  2009,  2001,  2241,
          2006,  2065,  2049,  3855,  2003,  2035,  2008,  2003,  2428,  2734,
          2005,  1996,  2364,  3720,  1

In [19]:
def compute_metrics(p):
  print(type(p))
  pred,label = p
  pred = np.argmax(pred,axis = 1)

  accuracy = accuracy_score(label,pred)
  precision = precision_score(label,pred)
  recall = recall_score(label,pred)
  f1 = f1_score(label,pred)

  return {"Accuracy" : accuracy , "precision" : precision , "Recall" : recall}

In [20]:
# Define trainer
args = TrainingArguments(
    output_dir = "output",
    num_train_epochs = 1,
    per_device_train_batch_size = 8
)

trainer = Trainer(
    model =model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics
)

In [21]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.3195821189880371, metrics={'train_runtime': 95.2689, 'train_samples_per_second': 8.397, 'train_steps_per_second': 1.05, 'total_flos': 210488844288000.0, 'train_loss': 0.3195821189880371, 'epoch': 1.0})

In [22]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.1738804429769516,
 'eval_Accuracy': 0.895,
 'eval_precision': 0.0,
 'eval_Recall': 0.0,
 'eval_runtime': 6.5692,
 'eval_samples_per_second': 30.445,
 'eval_steps_per_second': 3.806,
 'epoch': 1.0}

In [24]:
text = " That was good point"

inputs = tokenizer(text,padding= True,truncation = True,return_tensors="pt").to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits,dim = -1)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.6292, -2.2120]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


array([[0.97898406, 0.02101596]], dtype=float32)

In [25]:
trainer.save_model("custom_model")

In [26]:
model2 = BertForSequenceClassification.from_pretrained("custom_model")
model2.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [27]:
text = "go to hell"
inputs = tokenizer(text,padding = True,truncation = True,return_tensors="pt").to("cuda")
outputs = model2(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits,dim = -1)
predictions = predictions.cpu().detach().numpy()

In [28]:
predictions

array([[0.70275337, 0.29724663]], dtype=float32)